# Face Recognition Model Training

In [ ]:
!pip install tensorflow mtcnn scikit-learn
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
import cv2

In [ ]:
def load_dataset(data_dir='face_data'):
    images = []
    labels = []
    
    for person_name in os.listdir(data_dir):
        person_dir = os.path.join(data_dir, person_name)
        if os.path.isdir(person_dir):
            for img_name in os.listdir(person_dir):
                img_path = os.path.join(person_dir, img_name)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (160, 160))
                img = img / 255.0  # Normalize
                images.append(img)
                labels.append(person_name)
    
    return np.array(images), np.array(labels)

In [ ]:
# Load and prepare data
X, y = load_dataset()
le = LabelEncoder()
y = le.fit_transform(y)
num_classes = len(le.classes_)

# Save label encoder classes
np.save('face_classes.npy', le.classes_)

In [ ]:
# Create and train model
base_model = MobileNetV2(input_shape=(160, 160, 3), include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X, y, batch_size=32, epochs=10, validation_split=0.2)

# Save the model
model.save('face_recognition_model.h5')